In [2]:
# importing the required libraries

!pip install textstat
!pip install nrclex
!pip install chardet

import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.matutils import corpus2csc
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.coherencemodel import CoherenceModel

from wordcloud import WordCloud
import string
import matplotlib.pyplot as plt

from textstat import textstat
from nrclex import NRCLex

import chardet
import spacy

from collections import defaultdict
from textblob import TextBlob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of nrclex to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nrclex: filename=NRCLex-3.0.0-py3-none-any.whl size=43309 sha256=91f33f64bc872c1da5b42997005293fb6901cb3385792873ec2679a85728c78b
  Stored in directory: /root/.cache/pip/wheels/d2/10/44/6abfb1234298806a145fd6bcaec8cbc712e88dd1cd6cb242fa
Successfully built nrclex


In [4]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/MyDrive/Develop_Countries_Hydrogen_Fuel.csv', 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']
dataset = pd.read_csv('/content/drive/MyDrive/Develop_Countries_Hydrogen_Fuel.csv', encoding=encoding)
dataset.head()

Mounted at /content/drive


,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Total Interactions (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),Overperforming Score
0,Cat Electric Power,Caterpillar.Electric.Power,1.000650e+14,PRODUCT_SERVICE,US,Caterpillar and your local Cat® dealer are rea...,18-11-2008 19:51,213061.0,220357.0,2024-01-19 22:00:15 GMT,...,https://www.cat.com/en_US/news/engine-press-re...,https://www.cat.com/en_US/news/engine-press-re...,NaN,Caterpillar Demonstrates Viability of Using Hy...,Caterpillar Inc. announced the success of its ...,NaN,NaN,NaN,"1,69,157","3,759.04"
1,BullSnot,TheBullSnot,1.000640e+14,CHEMICALS_GASSES,US,"At Brown Ox Ventures, our goal is to provide o...",26-11-2015 01:38,5067.0,7888.0,2023-06-28 19:45:51 BST,...,https://www.facebook.com/photo.php?fbid=736196...,NaN,"‎'‎סס0, NE YEAR YEAR‎'‎",NaN,NaN,NaN,NaN,NaN,"1,11,433","1,466.22"
2,Hydrogen Cars Now,hydrogencarsnow,1.000570e+14,AUTO_COMPANY,US,"Hydrogen Cars Now reviews fuel cell vehicles, ...",28-11-2012 19:49,4906.0,6155.0,2023-11-21 18:55:18 GMT,...,https://www.facebook.com/photo.php?fbid=841505...,NaN,海日 HONDA EFUELCELL PSY NFUEL CEL,NaN,NaN,NaN,NaN,NaN,"96,967",828.78
3,Car Phanatics,carphanatics,1.000640e+14,CAR,US,We bring you the wonderful world of vehicles.,25-04-2019 10:30,52186.0,60854.0,2023-03-03 02:13:45 GMT,...,https://www.facebook.com/photo.php?fbid=578952...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"76,650",644.12
4,Heather Cox Richardson,heathercoxrichardson,1.000450e+14,AUTHOR,US,Heather Cox Richardson is a political historia...,11-04-2013 18:02,1278656.0,1725227.0,2023-07-07 04:51:55 BST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"53,519",-1.04


In [6]:
dataset['Combined Text'] = dataset['Message'].fillna('') + ' ' + dataset['Image Text'].fillna('') + ' ' + dataset['Link Text'].fillna('')
dataset.head()

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Total Interactions (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x ),Overperforming Score,Combined Text
0,Cat Electric Power,Caterpillar.Electric.Power,1.000650e+14,PRODUCT_SERVICE,US,Caterpillar and your local Cat® dealer are rea...,18-11-2008 19:51,213061.0,220357.0,2024-01-19 22:00:15 GMT,...,https://www.cat.com/en_US/news/engine-press-re...,NaN,Caterpillar Demonstrates Viability of Using Hy...,Caterpillar Inc. announced the success of its ...,NaN,NaN,NaN,"1,69,157","3,759.04",Can hydrogen power a data center for 48 hours?...
1,BullSnot,TheBullSnot,1.000640e+14,CHEMICALS_GASSES,US,"At Brown Ox Ventures, our goal is to provide o...",26-11-2015 01:38,5067.0,7888.0,2023-06-28 19:45:51 BST,...,NaN,"‎'‎סס0, NE YEAR YEAR‎'‎",NaN,NaN,NaN,NaN,NaN,"1,11,433","1,466.22","This is what 11,000 horsepower does to a top f..."
2,Hydrogen Cars Now,hydrogencarsnow,1.000570e+14,AUTO_COMPANY,US,"Hydrogen Cars Now reviews fuel cell vehicles, ...",28-11-2012 19:49,4906.0,6155.0,2023-11-21 18:55:18 GMT,...,NaN,海日 HONDA EFUELCELL PSY NFUEL CEL,NaN,NaN,NaN,NaN,NaN,"96,967",828.78,Honda Shows CR-V Hydrogen Fuel Cell Module Hon...
3,Car Phanatics,carphanatics,1.000640e+14,CAR,US,We bring you the wonderful world of vehicles.,25-04-2019 10:30,52186.0,60854.0,2023-03-03 02:13:45 GMT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"76,650",644.12,"What 10,000 horsepower does to a top fuel tire..."
4,Heather Cox Richardson,heathercoxrichardson,1.000450e+14,AUTHOR,US,Heather Cox Richardson is a political historia...,11-04-2013 18:02,1278656.0,1725227.0,2023-07-07 04:51:55 BST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"53,519",-1.04,"July 6, 2023 (Thursday) The payroll processing..."


In [8]:
data = dataset['Combined Text']
data

0       Can hydrogen power a data center for 48 hours?...
1       This is what 11,000 horsepower does to a top f...
2       Honda Shows CR-V Hydrogen Fuel Cell Module Hon...
3       What 10,000 horsepower does to a top fuel tire...
4       July 6, 2023 (Thursday) The payroll processing...
                              ...                        
4174    Our new Queensland SuperGrid ♻️ will connect s...
4175    BREAKING NEWS FROM REUTERS, MOTORTREND, HONDA,...
4176    This is how its done..  How To Build A “Water ...
4177    TODAY: Kangen Water® Party 12:30 - 2 p.m. (PST...
4178    To switch some (5-15% of the mix) of our gas t...
Name: Combined Text, Length: 4179, dtype: object

In [9]:
nlp = spacy.load("en_core_web_sm")

# lemmatization and removing stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower()
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in string.punctuation]
    words = [word for word in words if word not in stopwords.words("english")]
    doc = nlp(" ".join(words))
    words = [token.text for token in doc if token.ent_type_ == '' and token.text not in stopwords.words("english") and token.text not in string.punctuation]
    return words

data = data.apply(preprocess)
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0       [center, announces, significant, breakthrough,...
1       [horsepower, doe, top, fuel, dragster, tire, l...
2       [provided, detail, next, generation, fuel, cel...
3       [horsepower, doe, top, fuel, tire, launch, top...
4       [payroll, processing, firm, adp, said, private...
                              ...                        
4174    [supergrid, connect, solar, wind, pumped, hydr...
4175    [breaking, news, reuters, motortrend, honda, -...
4176    [done, .., build, “, water, powered, vehicle, ...
4177    [kangen, water, ®, party, pst, january, 4, 202...
4178    [switch, 15, mix, gas, home, hydrogen, switch,...
Name: Combined Text, Length: 4179, dtype: object

In [14]:
# Creating a dictionary from the preprocessed data
dictionary = Dictionary(data)

# Filtering out words that appear in fewer than 5 documents or more than 50% of the documents
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Initializing variables
best_coherence_score = -1
best_num_topics = 0
best_ldamodel = None
best_topics = []

# Storing all coherence scores and topics
all_coherence_scores = {}
all_ldamodels = {}
all_topics = {}

for num_topics in range(1, 16):
    print(f"\nNumber of Topics: {num_topics}")

    # Converting the data to bag of words format
    bow_corpus = [dictionary.doc2bow(text) for text in data]

    # Training the LDA model
    ldamodel = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=20, alpha='auto', eta='auto')

    # Calculating coherence score
    coherence_model = CoherenceModel(model=ldamodel, texts=data, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    all_coherence_scores[num_topics] = coherence_score
    all_ldamodels[num_topics] = ldamodel

    # Get the topics for this number of topics
    topics = ldamodel.show_topics(num_topics=num_topics, num_words=10, log=False, formatted=False)
    all_topics[num_topics] = topics

    print(f"Coherence Score: {coherence_score}")

    # Checking if the current coherence score is better than the previous best score
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_num_topics = num_topics
        best_ldamodel = ldamodel
        best_topics = topics

# Printing all topics
print("\nAll Topics:")
for num_topics, topics in all_topics.items():
    print(f"Number of Topics: {num_topics}")
    for topic_id, topic in topics:
        words_without_punctuations = [word for word, _ in topic if word.isalpha()]
        print("Topic:", topic_id)
        print("Words:", words_without_punctuations)

# Printing the number of topics with the best coherence score
print(f"\nNumber of Topics with Best Coherence Score: {best_num_topics}")
print(f"Best Coherence Score: {best_coherence_score}")

# Printing the best topics
print("Best Topics:")
for topic_id, topic in best_topics:
    words_without_punctuations = [word for word, _ in topic if word.isalpha()]  # Remove punctuations
    print("Topic:", topic_id)
    print("Words:", words_without_punctuations)


Number of Topics: 1


Coherence Score: 0.3431201184711434

Number of Topics: 2


Coherence Score: 0.3724206845411999

Number of Topics: 3


Coherence Score: 0.3642610985261985

Number of Topics: 4
Coherence Score: 0.40855339884415226

Number of Topics: 5
Coherence Score: 0.4111614050853946

Number of Topics: 6
Coherence Score: 0.39855426220124596

Number of Topics: 7
Coherence Score: 0.3793509668762047

Number of Topics: 8
Coherence Score: 0.41721890293701824

Number of Topics: 9
Coherence Score: 0.41374472982049476

Number of Topics: 10
Coherence Score: 0.42776804395950874

Number of Topics: 11
Coherence Score: 0.38622004513740366

Number of Topics: 12
Coherence Score: 0.43241369245759403

Number of Topics: 13
Coherence Score: 0.40819659015473236

Number of Topics: 14
Coherence Score: 0.4018672569379481

Number of Topics: 15
Coherence Score: 0.3823579417265048

All Topics:
Number of Topics: 1
Topic: 0
Words: ['energy', 'power', 'cell', 'http', 'electric', 'ha', 'vehicle', 'new']
Number of Topics: 2
Topic: 0
Words: ['cell', 'electric', 'vehicle', 'http', 'powered', 'car', 'truck', 'power']
Topic: 1
Words: ['energy', 'power

In [15]:
topic_names = {
    0: "Rocket Launches and Space Exploration",
    1: "Astronomy and Celestial Phenomena",
    2: "Electric Vehicles and Nikola Motors",
    3: "Power and Health Performance",
    4: "Nuclear Fusion and Water Power",
    5: "Battery Technology and Toxicity",
    6: "Renewable Energy and Carbon Reduction",
    7: "Electric Aircraft and Aviation",
    8: "Future Clean Energy Technologies",
    9: "Gas Products and Visionary Concepts",
    10: "Government Policies and State Initiatives",
    11: "Electric Cars and Sustainable Transportation"
}

# Print named topics
print("\nNamed Topics:")
for topic_id, topic_words in best_topics:
    topic_name = topic_names.get(topic_id, "Unnamed Topic")
    print("Topic:", topic_id, "-", topic_name)
    print("Words:", topic_words)


Named Topics:
Topic: 0 - Rocket Launches and Space Exploration
Words: [('launch', 0.028681811), ('rocket', 0.026048893), ("'s", 0.02109947), ('leak', 0.019180387), ('space', 0.01810848), ('fire', 0.0154669015), ('wa', 0.0152497655), ('flight', 0.013405062), ('liquid', 0.012887885), ('moon', 0.012438705)]
Topic: 1 - Astronomy and Celestial Phenomena
Words: [('star', 0.025595576), ('light', 0.014838641), ('water', 0.013890832), ('sun', 0.0126800295), ("'s", 0.009735545), ('life', 0.009065726), ('carbon', 0.008575892), ('image', 0.007865469), ('high', 0.00744901), ('also', 0.0070713386)]
Topic: 2 - Electric Vehicles and Nikola Motors
Words: [('cell', 0.037936516), ('electric', 0.026195373), ("'s", 0.022545451), ('http', 0.020777084), ('vehicle', 0.018286379), ('engine', 0.016820291), ('ha', 0.0155511405), ('powered', 0.014955243), ('car', 0.013060204), ('nikola', 0.010964802)]
Topic: 3 - Power and Health Performance
Words: [("'s", 0.034392346), ('power', 0.0210303), ('food', 0.020228934)

In [16]:
# importing the required libraries
nltk.download('punkt')
nltk.download('wordnet')

# Function to compute sentiment score for a word based on its sentiment in WordNet
def get_word_sentiment_score(word):
    synsets = wn.synsets(word)
    if synsets:
        synset = synsets[0]
        return TextBlob(synset.definition()).sentiment.polarity
    else:
        return 0

# Function to compute topic valence based on sentiment scores of words
def compute_topic_valence(topic_words):
    sentiment_scores = [get_word_sentiment_score(word) for word in topic_words]
    print("Sentiment scores:", sentiment_scores)
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)
    return avg_sentiment

document_topics_distribution = defaultdict(list)

for index, document in enumerate(data):
    bow_vector = dictionary.doc2bow(document)
    topic_distribution = best_ldamodel.get_document_topics(bow_vector, minimum_probability=0.0)
    document_topics_distribution[index] = topic_distribution

best_topic_valence = {}

# Computing the topic valence for each of the best topics
for topic_id, topic in best_topics:
    words = [word for word, _ in topic if word.isalpha()]
    topic_valence = compute_topic_valence(words)
    best_topic_valence[topic_id] = topic_valence

document_valence_scores = defaultdict(list)

# Computing the valence score for each document for each best topic
for index, topic_distribution in document_topics_distribution.items():
    for topic_id, _ in best_topics:
        topic_valence = best_topic_valence[topic_id]
        valence_score = 0
        for tid, probability in topic_distribution:
            if tid == topic_id:
                valence_score += topic_valence * probability
        document_valence_scores[index].append(valence_score)

df_document_valence = pd.DataFrame.from_dict(document_valence_scores, orient='index', columns=[f'Topic_{i}_Valence' for i in range(1, best_num_topics+1)])
print(df_document_valence)

# Calculating the topic distribution
document_topic_distribution_scores = defaultdict(list)

# Computing the topic distribution scores for each document for each best topic
for index, topic_distribution in document_topics_distribution.items():
    for topic_id, _ in best_topics:
        distribution_score = 0
        for tid, probability in topic_distribution:
            if tid == topic_id:
                distribution_score = probability
        document_topic_distribution_scores[index].append(distribution_score)

df_document_topic_distribution = pd.DataFrame.from_dict(document_topic_distribution_scores, orient='index', columns=[f'Topic_{i}_Distribution' for i in range(1, best_num_topics+1)])
print(df_document_topic_distribution)
df_combined = pd.concat([df_document_topic_distribution, df_document_valence], axis=1)
merged_df = dataset.merge(df_combined, left_index=True, right_index=True)
merged_df['FOG_Score'] = merged_df['Combined Text'].apply(textstat.gunning_fog)
def analyze_emotions(text):
    emotions = NRCLex(text)
    emotion_scores = emotions.raw_emotion_scores
    return emotion_scores
merged_df['NRC_Emotions'] = merged_df['Combined Text'].apply(analyze_emotions)
merged_df.to_csv('Developed_Countries_Final.csv', index=False)

from google.colab import files
files.download('Developed_Countries_Final.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Sentiment scores: [-0.08333333333333333, 0.0, 0.2, 0.0, -0.6, 0.0, 0.0, 0.0, 0.1]
Sentiment scores: [0.25, 0.0, -0.15, 0.4, -0.06666666666666667, 0.6, 0.2, 0.0, 0.0]
Sentiment scores: [-0.25, 0.0, 0.0, 0.0, 0.0, 0.22619047619047616, 0.0, -0.125, 0]
Sentiment scores: [0.0, 0.0, -0.21666666666666665, 0.0, 0.45, 0.13333333333333333, 0.0]
Sentiment scores: [-0.15, -0.6, 0.0, 0.0, 0, 0.0, 0.0, -0.8, 0.0]
Sentiment scores: [0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, -0.125, 0.13749999999999998]
Sentiment scores: [0.0, 0.0, -0.2, 0.0, 0.32, 0.0, 0.0, 0.6, 0.08666666666666667]
Sentiment scores: [0.8, 0.1, -0.1, 0.0, 0.0, 0.3, 0]
Sentiment scores: [0.0, 0.0, 0.0, -0.25, 0.0, -0.025, 0.55, 0.0, 0.0, 0.0]
Sentiment scores: [0.0, 0.32, -0.15, 0.0, 0.012499999999999997, 0.0, 0.0, 0.02500000000000001]
Sentiment scores: [0.0, 0.22619047619047616, 0.0, 0.0, 0.0, 0.03333333333333333]
Sentiment scores: [0.0, 0.0, -0.125, 0.0, -0.25, -0.025, 0.0, 0.0, 0.0, 0.32]
      Topic_1_Valence  Topic_2_Valence  Topic_3_V

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>